In [337]:
import torch
import numpy as np
import pandas as pd
import os
import sys

In [338]:
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from torch import optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap

In [339]:
from scipy import stats

## Read Data

In [340]:
data = pd.read_csv('../data/gesture-recognition-and-biometrics-electromyogram-grabmyo-1.0.2/features_v2.csv')

In [364]:
def simple_model(input_shape, output_shape):
    model = nn.Sequential(
        nn.Linear(input_shape, 64),  # Input layer: Fully connected (linear) with 64 units
        nn.ReLU(),  # Activation function: ReLU,
        nn.Dropout(0.3),
        nn.Linear(64, 128),
        nn.ReLU(),  # Activation function: ReLU
        nn.Dropout(0.3),
        # nn.Linear(128, 128),
        # nn.ReLU(),  # Activation function: ReLU
        # nn.Dropout(0.3),
        nn.Linear(128, output_shape)  # Output layer: Fully connected (linear) with 'output_shape' units
    )
    
    return model

In [365]:
data.head()

,session,trial,filename,crest_factor_F1,dasd_F1,diffvar_F1,form_factor_F1,iemg_F1,kurtosis_F1,kurtosis_f_F1,...,rms_W9,skew_W9,skew_f_W9,ssi_W9,sum_f_W9,var_f_W9,wflen_W9,willison_W9,gesture,participant
0,1,1,E:\DS5500-project\data\gesture-recognition-and...,7.091099,0.021834,0.000606,-6502.754507,365.456658,2.574033,85.547962,...,0.022588,-0.148877,5.529415,5.224663,0.109574,2.901826e-10,69.944068,15.0,10,1
1,1,2,E:\DS5500-project\data\gesture-recognition-and...,13.324514,0.023852,0.000689,1352.897349,379.664639,16.942218,329.114870,...,0.016172,-0.205427,9.332349,2.677940,0.077566,2.769772e-10,54.530919,23.0,10,1
2,1,3,E:\DS5500-project\data\gesture-recognition-and...,8.047954,0.016213,0.000326,1295.052394,296.823135,3.795827,134.751765,...,0.017039,-0.204604,9.078430,2.973002,0.064458,1.426359e-10,57.815909,13.0,10,1
3,1,4,E:\DS5500-project\data\gesture-recognition-and...,8.459555,0.026572,0.000864,14453.068135,479.688214,6.938354,98.832790,...,0.031982,-0.266818,5.067522,10.473811,0.131909,4.509337e-10,83.441138,0.0,10,1
4,1,5,E:\DS5500-project\data\gesture-recognition-and...,8.329011,0.026896,0.000901,11827.310277,464.350257,2.379207,116.412205,...,0.031134,-0.092205,5.414199,9.925787,0.177516,9.577100e-10,94.393506,3.0,10,1


In [366]:
feature_cols = [c for c in data.columns if "_" in c]

In [367]:
new_data = data[(np.abs(stats.zscore(data.loc[:, feature_cols])) < 5.5).all(axis = 1)].reset_index()

In [368]:
stats.zscore(data.loc[:, feature_cols])

,crest_factor_F1,dasd_F1,diffvar_F1,form_factor_F1,iemg_F1,kurtosis_F1,kurtosis_f_F1,mav_F1,max_F1,max_f_F1,...,power_W9,pulse_indicator_W9,rms_W9,skew_W9,skew_f_W9,ssi_W9,sum_f_W9,var_f_W9,wflen_W9,willison_W9
0,0.610386,0.188518,-0.069701,-0.008071,-0.050131,0.047544,0.057123,-0.050131,-0.040325,-0.077027,...,-0.238716,0.008071,-0.412892,0.325930,0.224663,-0.238716,-0.055548,-0.102397,-0.279489,0.070692
1,4.254780,0.336284,-0.003999,-0.008071,0.002687,1.772062,1.367057,0.002687,0.219375,0.675523,...,-0.286458,0.008071,-0.663671,0.218519,1.765068,-0.286458,-0.232764,-0.103586,-0.535182,0.276470
2,1.169816,-0.223075,-0.292047,-0.008071,-0.305275,0.194188,0.321747,-0.305275,-0.272018,-0.130563,...,-0.280926,0.008071,-0.629761,0.220081,1.662216,-0.280926,-0.305333,-0.115684,-0.480686,0.019248
3,1.410460,0.535428,0.135751,-0.008071,0.374524,0.571365,0.128571,0.374524,0.526030,0.311826,...,-0.140312,0.008071,-0.045758,0.101910,0.037569,-0.140312,0.068112,-0.087922,-0.055583,-0.315142
4,1.334137,0.559166,0.164994,-0.008071,0.317505,0.024160,0.223115,0.317505,0.311552,0.173976,...,-0.150586,0.008071,-0.078898,0.433574,0.177993,-0.150586,0.320617,-0.042286,0.126108,-0.237975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15346,1.008158,-0.443099,-0.386205,-0.008071,-0.390027,-0.000158,-0.247598,-0.390027,-0.385235,-0.316472,...,-0.255784,0.008071,-0.493494,0.051025,-0.463577,-0.255784,-0.466860,-0.126757,-0.760141,-0.263697
15347,1.827370,-0.009144,-0.210208,-0.008071,0.025106,0.292572,0.313602,0.025106,0.064689,-0.179573,...,-0.208629,0.008071,-0.286366,-0.131926,-0.313764,-0.208629,-0.240535,-0.119488,-0.571487,-0.315142
15348,-0.072481,-0.053205,-0.226733,-0.008071,0.037538,-0.031888,-0.281347,0.037538,0.031521,-0.268886,...,-0.191909,0.008071,-0.222484,-0.001042,-0.242612,-0.191909,-0.213713,-0.118389,-0.447048,-0.315142
15349,0.245131,-0.257046,-0.319502,-0.008071,-0.220962,0.001354,-0.285374,-0.220962,-0.211510,-0.307252,...,-0.302404,0.008071,-0.773610,-0.328416,-0.462758,-0.302404,-0.314849,-0.122826,-0.844164,-0.315142


## split on Participants

In [369]:
rand_parts =  np.random.choice(data.participant.unique(), 5, replace = False)

In [370]:
rand_parts = np.array([2,4,13,25,39])

In [245]:
rand_parts = list(set(range(1,44)) - set([29,20,26,23,4,34, 6, 33, 10, 39,3, 28, 21, 42, 36, 11, 
                        19, 18, 7, 40, 22, 41,  5, 12, 8, 15, 37, 16, 
                        35,  2, 43, 17, 32, 24 , 9, 30, 31, 27]))

In [371]:
rand_parts

array([ 2,  4, 13, 25, 39])

In [372]:
train_df = new_data[~new_data['participant'].isin(rand_parts)]
test_df = new_data[new_data['participant'].isin(rand_parts)]

In [373]:
train_df

,index,session,trial,filename,crest_factor_F1,dasd_F1,diffvar_F1,form_factor_F1,iemg_F1,kurtosis_F1,...,rms_W9,skew_W9,skew_f_W9,ssi_W9,sum_f_W9,var_f_W9,wflen_W9,willison_W9,gesture,participant
0,0,1,1,E:\DS5500-project\data\gesture-recognition-and...,7.091099,0.021834,0.000606,-6502.754507,365.456658,2.574033,...,0.022588,-0.148877,5.529415,5.224663,0.109574,2.901826e-10,69.944068,15.0,10,1
1,1,1,2,E:\DS5500-project\data\gesture-recognition-and...,13.324514,0.023852,0.000689,1352.897349,379.664639,16.942218,...,0.016172,-0.205427,9.332349,2.677940,0.077566,2.769772e-10,54.530919,23.0,10,1
2,2,1,3,E:\DS5500-project\data\gesture-recognition-and...,8.047954,0.016213,0.000326,1295.052394,296.823135,3.795827,...,0.017039,-0.204604,9.078430,2.973002,0.064458,1.426359e-10,57.815909,13.0,10,1
3,3,1,4,E:\DS5500-project\data\gesture-recognition-and...,8.459555,0.026572,0.000864,14453.068135,479.688214,6.938354,...,0.031982,-0.266818,5.067522,10.473811,0.131909,4.509337e-10,83.441138,0.0,10,1
4,4,1,5,E:\DS5500-project\data\gesture-recognition-and...,8.329011,0.026896,0.000901,11827.310277,464.350257,2.379207,...,0.031134,-0.092205,5.414199,9.925787,0.177516,9.577100e-10,94.393506,3.0,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13010,15346,3,3,E:\DS5500-project\data\gesture-recognition-and...,7.771453,0.013208,0.000208,2599.892725,274.024844,2.176596,...,0.020526,-0.293608,3.830298,4.314177,0.035283,1.966555e-11,40.970398,2.0,9,9
13011,15347,3,4,E:\DS5500-project\data\gesture-recognition-and...,9.172642,0.019135,0.000429,1255.242161,385.695263,4.615531,...,0.025825,-0.389927,4.200155,6.829604,0.076162,1.003953e-10,52.342457,0.0,9,9
13012,15348,3,5,E:\DS5500-project\data\gesture-recognition-and...,5.923115,0.018533,0.000408,1713.228290,389.039490,1.912226,...,0.027460,-0.321020,4.375813,7.721463,0.081006,1.125954e-10,59.843660,0.0,9,9
13013,15349,3,6,E:\DS5500-project\data\gesture-recognition-and...,6.466362,0.015749,0.000292,-992.810528,319.503197,2.189195,...,0.013359,-0.493375,3.832319,1.827333,0.062739,6.332731e-11,35.905471,0.0,9,9


In [374]:
test_df

,index,session,trial,filename,crest_factor_F1,dasd_F1,diffvar_F1,form_factor_F1,iemg_F1,kurtosis_F1,...,rms_W9,skew_W9,skew_f_W9,ssi_W9,sum_f_W9,var_f_W9,wflen_W9,willison_W9,gesture,participant
378,476,1,1,E:\DS5500-project\data\gesture-recognition-and...,5.208832,0.017824,0.000406,3575.240018,309.164606,1.018654,...,0.033558,-0.393727,3.994849,11.531936,0.064356,6.996360e-11,105.934448,12.0,10,13
379,477,1,2,E:\DS5500-project\data\gesture-recognition-and...,5.361524,0.023917,0.000725,-891.240580,416.721611,1.172301,...,0.049668,-0.430486,4.459720,25.260854,0.136629,3.976704e-10,148.030136,10.0,10,13
380,478,1,3,E:\DS5500-project\data\gesture-recognition-and...,4.917588,0.017482,0.000389,13458.721887,312.542667,1.028537,...,0.039848,-0.585121,6.967857,16.259969,0.074714,1.616857e-10,114.383120,28.0,10,13
381,479,1,4,E:\DS5500-project\data\gesture-recognition-and...,6.074335,0.017875,0.000421,-3307.478906,307.580026,1.609005,...,0.042370,-0.751108,5.821305,18.382626,0.088132,2.331914e-10,126.547165,27.0,10,13
382,480,1,5,E:\DS5500-project\data\gesture-recognition-and...,5.996519,0.016348,0.000343,-561.850306,284.522972,0.864712,...,0.033749,-0.439346,3.830835,11.663535,0.054747,4.669696e-11,104.165571,1.0,10,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12099,14274,3,2,E:\DS5500-project\data\gesture-recognition-and...,5.365565,0.012542,0.000178,725.317181,364.290833,0.694718,...,0.069551,-0.239551,3.637887,49.534604,0.070771,8.229783e-11,162.850710,0.0,9,4
12100,14275,3,3,E:\DS5500-project\data\gesture-recognition-and...,6.019943,0.014517,0.000248,608.759589,362.410204,1.117007,...,0.066124,-0.246990,3.759432,44.773385,0.073940,9.265135e-11,155.776720,0.0,9,4
12101,14276,3,4,E:\DS5500-project\data\gesture-recognition-and...,5.560879,0.009745,0.000107,604.489325,262.513309,1.998836,...,0.065269,-0.213617,4.971597,43.622275,0.052794,7.486254e-11,140.487093,0.0,9,4
12102,14277,3,5,E:\DS5500-project\data\gesture-recognition-and...,4.817575,0.011708,0.000158,1781.837538,312.025253,0.987789,...,0.070486,-0.137336,3.296031,50.875813,0.067907,7.629225e-11,170.308322,0.0,9,4


In [375]:
X = train_df.loc[:, feature_cols].values
Y = (train_df.loc[:, 'gesture']-1).values

In [376]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [377]:
x_test = test_df.loc[:, feature_cols].values
y_test = (test_df.loc[:, 'gesture']-1).values
x_test = scaler.transform(x_test)

In [378]:
train_df.gesture.unique()

array([10, 11, 12, 13, 14, 15, 16, 17,  1,  2,  3,  4,  5,  6,  7,  8,  9],
      dtype=int64)

In [379]:
test_df.gesture.unique()

array([10, 11, 12, 13, 14, 15, 16, 17,  1,  2,  3,  4,  5,  6,  7,  8,  9],
      dtype=int64)

In [380]:
max(train_df.loc[:, feature_cols]['form_factor_F1'])

3656057.702453692

In [381]:
# x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=56)

In [382]:
train_dataset = TensorDataset(torch.tensor(X).type(torch.float32), torch.tensor(Y).type(torch.LongTensor))
test_dataset = TensorDataset(torch.tensor(x_test).type(torch.float32), torch.tensor(y_test).type(torch.LongTensor))

In [383]:
# Define batch size and whether to shuffle the data
batch_size = 256
shuffle = True

# Create data loaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [384]:
model = simple_model(x_train.shape[1], len(np.unique(y_train)))

In [385]:
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=5e-4)

In [386]:
num_epochs = 60

In [387]:
# Training loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    total_train_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    # Validation phase
    model.eval()
    total_val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(test_loader):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate and log metrics
    train_loss = total_train_loss / len(train_loader)
    val_loss = total_val_loss / len(test_loader)
    val_accuracy = 100 * correct / total

    if (epoch+1)%20==0:
        print(f"Epoch [{epoch + 1}/{num_epochs}] - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

Epoch [20/60] - Train Loss: 0.5256, Val Loss: 0.8950, Val Accuracy: 71.35%
Epoch [40/60] - Train Loss: 0.3726, Val Loss: 0.9389, Val Accuracy: 73.27%
Epoch [60/60] - Train Loss: 0.2938, Val Loss: 1.0195, Val Accuracy: 74.10%


In [388]:
def predict(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    y_pred = []

    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(test_loader):
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_pred.extend(predicted.tolist())

    return y_pred

In [389]:
tune_participant = rand_parts[0]

In [390]:
tune_participant

2

In [391]:
test_df.participant.unique()

array([13,  2, 25, 39,  4], dtype=int64)

In [392]:
tune_data = test_df[test_df['participant'] == tune_participant]

In [393]:
tune_data.gesture.unique()

array([10, 12, 13, 14, 15, 16, 17,  1,  2,  3,  4,  5,  6,  7,  8,  9, 11],
      dtype=int64)

In [394]:
X_tune = tune_data.loc[:, feature_cols].values
Y_tune = (tune_data.loc[:, 'gesture']-1).values

In [395]:
X_tune = scaler.transform(X_tune)

In [396]:
test_tune_ds = TensorDataset(torch.tensor(X_tune).type(torch.float32), torch.tensor(Y_tune).type(torch.LongTensor))

In [397]:
test_tune_loader = DataLoader(test_tune_ds, batch_size=batch_size, shuffle=False)

In [398]:
x_tune_pred = predict(model, test_tune_loader)

In [399]:
print(classification_report(Y_tune, x_tune_pred))

              precision    recall  f1-score   support

           0       0.62      0.75      0.68        20
           1       0.67      0.10      0.17        21
           2       0.39      0.65      0.49        20
           3       1.00      0.10      0.17        21
           4       0.56      0.95      0.71        19
           5       0.67      0.48      0.56        21
           6       0.47      0.37      0.41        19
           7       0.64      0.47      0.55        19
           8       0.44      0.75      0.56        20
           9       0.86      0.90      0.88        20
          10       0.40      1.00      0.57         2
          11       0.81      0.93      0.87        14
          12       0.47      1.00      0.64         7
          13       0.67      1.00      0.80        12
          14       0.86      0.63      0.73        19
          15       0.71      0.56      0.63        18
          16       0.88      0.74      0.80        19

    accuracy              

In [400]:
tuning_df = test_df[(test_df['participant'] == tune_participant)&(test_df['trial'].isin([1,2,3]))]
tuning_test_df = test_df[(test_df['participant'] == tune_participant)&(~test_df['trial'].isin([1,2,3]))]

In [402]:
tuning_df['participant'].unique(), tuning_df['trial'].unique()

(array([2], dtype=int64), array([1, 2, 3], dtype=int64))

In [329]:
tuning_test_df

,index,session,trial,filename,crest_factor_F1,dasd_F1,diffvar_F1,form_factor_F1,iemg_F1,kurtosis_F1,...,rms_W9,skew_W9,skew_f_W9,ssi_W9,sum_f_W9,var_f_W9,wflen_W9,willison_W9,gesture,participant
1200,1550,1,4,E:\DS5500-project\data\gesture-recognition-and...,4.796877,0.023714,0.000716,20600.189169,423.303588,0.726222,...,0.048222,-0.482559,4.161127,23.812045,0.138653,3.215365e-10,140.445361,1.0,10,21
1201,1551,1,5,E:\DS5500-project\data\gesture-recognition-and...,4.322362,0.022856,0.000665,-441.868957,418.137637,0.453902,...,0.073931,-0.552093,3.261645,55.969481,0.175094,4.527577e-10,166.703374,0.0,10,21
1202,1553,1,7,E:\DS5500-project\data\gesture-recognition-and...,6.456900,0.017126,0.000362,-402.870845,338.534154,1.190239,...,0.066682,-0.306180,3.426416,45.532193,0.207924,7.098282e-10,172.834616,0.0,10,21
1205,1557,1,4,E:\DS5500-project\data\gesture-recognition-and...,6.644512,0.043332,0.002282,-1772.432378,838.422037,3.302728,...,0.040730,-0.306688,5.512830,16.987726,0.526905,9.326775e-09,143.133647,19.0,11,21
1206,1558,1,5,E:\DS5500-project\data\gesture-recognition-and...,5.131051,0.033517,0.001347,1569.259384,729.725685,1.342680,...,0.049438,-0.459699,4.564450,25.027552,0.438808,5.082834e-09,143.289534,7.0,11,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10021,11892,3,7,E:\DS5500-project\data\gesture-recognition-and...,5.338859,0.010056,0.000123,-925.089079,206.338488,1.050245,...,0.043571,-0.902904,5.410149,19.439798,0.044315,5.257060e-11,88.748840,0.0,8,21
10025,11896,3,4,E:\DS5500-project\data\gesture-recognition-and...,6.540888,0.011772,0.000170,5143.471248,231.663855,0.963000,...,0.025039,-0.528076,4.671460,6.419731,0.043681,3.633275e-11,57.953227,3.0,9,21
10026,11897,3,5,E:\DS5500-project\data\gesture-recognition-and...,8.528665,0.008750,0.000093,830.117004,180.545904,6.299686,...,0.020518,-0.211011,7.719600,4.311065,0.032775,2.687450e-11,47.086254,8.0,9,21
10027,11898,3,6,E:\DS5500-project\data\gesture-recognition-and...,5.980864,0.006746,0.000057,707.974400,122.625554,0.853298,...,0.006508,0.147181,5.690214,0.433767,0.021730,1.265072e-11,28.279331,39.0,9,21


In [403]:
X_tune = tuning_df.loc[:, feature_cols].values
Y_tune = (tuning_df.loc[:, 'gesture']-1).values

X_tune_test = tuning_test_df.loc[:, feature_cols].values
Y_tune_test = (tuning_test_df.loc[:, 'gesture']-1).values

In [404]:
X_tune, X_tune_test = scaler.transform(X_tune), scaler.transform(X_tune_test)

In [405]:
tune_ds = TensorDataset(torch.tensor(X_tune).type(torch.float32), torch.tensor(Y_tune).type(torch.LongTensor))
test_tune_ds = TensorDataset(torch.tensor(X_tune_test).type(torch.float32), torch.tensor(Y_tune_test).type(torch.LongTensor))

tune_loader = DataLoader(tune_ds, batch_size=batch_size, shuffle=False)
test_tune_loader = DataLoader(test_tune_ds, batch_size=batch_size, shuffle=False)

In [406]:
num_tune_epochs = 20

In [407]:
#tuning
# Training loop
for epoch in range(num_tune_epochs):
    # Training phase
    model.train()
    total_train_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(tune_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    # Validation phase
    model.eval()
    total_val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(test_tune_loader):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate and log metrics
    train_loss = total_train_loss / len(train_loader)
    val_loss = total_val_loss / len(test_loader)
    val_accuracy = 100 * correct / total

    if (epoch+1)%20==0:
        print(f"Epoch [{epoch + 1}/{num_epochs}] - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

Epoch [20/60] - Train Loss: 0.0067, Val Loss: 0.0422, Val Accuracy: 85.98%


In [408]:
pred_tune = predict(model, test_tune_loader)

In [409]:
print(classification_report(Y_tune_test, pred_tune))

              precision    recall  f1-score   support

           0       0.56      0.83      0.67        12
           1       0.91      0.83      0.87        12
           2       0.91      0.91      0.91        11
           3       1.00      0.75      0.86        12
           4       0.92      1.00      0.96        11
           5       0.71      0.83      0.77        12
           6       0.82      0.90      0.86        10
           7       0.83      0.91      0.87        11
           8       1.00      0.92      0.96        12
           9       0.89      0.73      0.80        11
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         7
          12       1.00      1.00      1.00         1
          13       1.00      1.00      1.00         7
          14       1.00      1.00      1.00        11
          15       0.71      0.42      0.53        12
          16       0.92      1.00      0.96        11

    accuracy              

### Scratch work

In [4]:
a = pd.DataFrame({"a":[1,2,3,4,5], 
     "b":[3,5,3,5,2], 
     "c":[6,2,3,7,5]})

In [6]:
def f(a):
    a = a[a['a'] < 3]

    return a

In [7]:
b = f(a)

In [8]:
b

,a,b,c
0,1,3,6
1,2,5,2


In [9]:
a

,a,b,c
0,1,3,6
1,2,5,2
2,3,3,3
3,4,5,7
4,5,2,5
